<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mortgage-class" data-toc-modified-id="Mortgage-class-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mortgage class</a></span></li><li><span><a href="#30-year-fixed-rate-mortgage" data-toc-modified-id="30-year-fixed-rate-mortgage-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>30 year fixed-rate mortgage</a></span></li><li><span><a href="#20-year-fixed-rate-mortgage" data-toc-modified-id="20-year-fixed-rate-mortgage-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>20 year fixed-rate mortgage</a></span></li><li><span><a href="#30-year-ARM" data-toc-modified-id="30-year-ARM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>30 year ARM</a></span></li></ul></div>

### Mortgage class

A python class "Mortgage", that can serve in creating the **amortization schedule** for both *fixed* and *adjusted rate mortgages* is created.

The class works by the following way:

* Creates a new instance of mortgage every time it is assigned to a variable.
The parameters that the Mortgage class takes are:
        
        * Principal amount
        * Interest rate
        * Period of mortgage in years
      
    In case of ARM, three additional parameters should be passed:
        
        * Variable years which is the total number of years when the reset will occur.
        * Years for reset (1 if the rates are reset every year, 0.5 if the rates are reset every six months)
        * Reset rates, which is a python list containing interest rates for the reset. The number of elements in the list 
          must be equal to the variable years.
          
    The class initiates four python lists. All the values of the list are in dollars.

        * Payment
        * Principal
        * Interest
        * Balance
          
* The method MakePayment() computes the payment using the principal amount, rate and the period. 
It does the following to the lists:

        * Appends a single payment to the payment list
        * Computes the interest amount paid and appends to interest list
        * Subtracts the interest from the payment and appends the value to principal list.
        * Subtracts the principal from previous balance and appends to the balance list.
       
* The method AmortizationSchedule() returns a Pandas DataFrame, containing five features.

        * Period
        * Payment
        * Principal
        * Interest
        * Balance
        
A *for loop* with the range of the total number of payments of the mortgage can be used to amortize the schedule to zero.

        Example:
        
            mort_a = Mortgage(1000000, 0.04, 30)
            for i in range(30*12):
                mort_a.MakePayment()    
            mort_a.AmortizationSchedule()

In [1]:
import pandas as pd
import altair as alt
pd.options.display.float_format = '{:.2f}'.format

class Mortgage(object):
    '''
    Creates an instance of class Mortgage. 
    Multi-purpose - can be used to schedule Fixed rate and adjustable rate mortgages.
    
    Parameters:
        loan: Initial loan amount
        interest_rate: Annual interest rate. 
                       If adjustable rate mortgage, pass the fixed term rate. Ex.,annual rate for 7 years for a 7-1 ARM
        years: Period of mortgage in years
        
        Optional (To be used for Adjusted rate mortgage)
        variable_years: Total years when the rates are adjusted. Ex., 23 <30-7> for 7-1 ARM amortizing in 30 years
        reset_time: Reset time in years. Ex., 1 for 7-1 ARM
        reset_rates: Assumes a list of interest rates for the remaining years of an adjustable mortgage
        
    Methods:
        MakePayment(): Adds one payment to the Amortization schedule
        GetPayments(): Returns the completed payments
        GetPrincipal(): Returns the paid principal each month
        GetInterest(): Returns the paid interest each month
        GetTotalPayments(): Returns the sum of completed payments
        GetBalance(): Returns the outstanding balance
        AmortizationSchedule(): Returns a Pandas DataFrame
    '''
    def __init__(self, loan, interest_rate, years, variable_years=0, reset_time=0, reset_rates=[]):
        if variable_years != 0:
            try:
                assert len(reset_rates) == variable_years/reset_time
            except:
                raise ValueError("The number of reset rates does not match the variable years")
        
        self.loan = loan
        self.rate = interest_rate/12
        self.months = years*12
        self.variable_years = variable_years
        self.fixed_years = years - variable_years
        self.reset_time = reset_time
        self.reset_rates = reset_rates
        self.paid = [0]
        self.principal = [0]
        self.interest = [0]        
        self.balance = [loan]
        
    def CalculatePayment(self, loan, monthly_rate, months):
        '''
        Assumes loan, monthly_rate and months to be int. Returns the monthly payment.
        '''
        num = loan*monthly_rate*(1+monthly_rate)**months
        den = (1+monthly_rate)**months - 1
        return num/den
    
    def MakePayment(self):
        '''
        Adds one payment to the mortgage schedule when invoked.
        '''
        if len(self.balance)-1 < self.fixed_years*12:
            Mortgage.interest_rate = self.rate
            Mortgage.pmt = Mortgage.CalculatePayment(self, self.loan, Mortgage.interest_rate, self.months)

        elif (len(self.balance)-1) % (12*self.reset_time) == 0:
            reset = int(((len(self.balance) - 1) - (self.fixed_years * 12)) / (12*self.reset_time))
            Mortgage.interest_rate = self.reset_rates[reset]/12
            bal = self.balance[-1]
            months = self.months - (12*self.fixed_years) - (12*self.reset_time*reset)
            Mortgage.pmt = Mortgage.CalculatePayment(self, bal, Mortgage.interest_rate, months)
        
        self.paid.append(Mortgage.pmt)
        interest = self.balance[-1] * Mortgage.interest_rate
        self.interest.append(interest)
        principal = Mortgage.pmt - interest
        self.principal.append(principal)
        reduction = self.balance[-1] - principal
        self.balance.append(reduction)
        
    def GetPayments(self):
        '''
        Returns a list of completed payments.
        '''
        return self.paid.copy()
    
    def GetPrinipal(self):
        '''
        Returns a list of Principal paid each month.
        '''
        return self.principal.copy()
    
    def GetInterest(self):
        '''
        Returns a list of interest paid each month.
        '''
        return self.interest.copy()
    
    def GetTotalPayments(self):
        '''
        Returns the sum of completed payments.
        '''
        return sum(self.paid)
    
    def GetUnpaid(self):
        '''
        Returns a list of unpaid balances on each month.
        '''
        return self.balance.copy()
    
    def AmortizationSchedule(self):
        '''
        Returns a Pandas DataFrame of the Amortization schedule 
        containing features Period, Payments, Principal, Interest and Unpaid Balance.
        '''
        return pd.DataFrame(
            {
                "Period": range(len(self.balance)),
                "Payments": self.GetPayments(),
                "Principal": self.GetPrinipal(),
                "Interest": self.GetInterest(),
                "Unpaid Balance": self.GetUnpaid()
            }
        ).apply(lambda x: round(x,2))

### 30 year fixed-rate mortgage

In [2]:
p = 66500
i = 0.275
n = 1

mortgage_a = Mortgage(p, i, n)

for i in range(n*12):
    mortgage_a.MakePayment()
    
schedule_a = mortgage_a.AmortizationSchedule()

schedule_a

,Period,Payments,Principal,Interest,Unpaid Balance
0,0,0.00,0.00,0.00,66500.00
1,1,6401.39,4877.43,1523.96,61622.57
2,2,6401.39,4989.21,1412.18,56633.36
3,3,6401.39,5103.54,1297.85,51529.81
4,4,6401.39,5220.50,1180.89,46309.31
5,5,6401.39,5340.14,1061.26,40969.18
6,6,6401.39,5462.51,938.88,35506.66
7,7,6401.39,5587.70,813.69,29918.97
8,8,6401.39,5715.75,685.64,24203.22
9,9,6401.39,5846.73,554.66,18356.48


In [3]:
schedule_a_bal = schedule_a.copy()
schedule_a_bal["Interest balance"] = schedule_a_bal["Interest"].sum() - schedule_a_bal["Interest"].cumsum()
schedule_a_bal["Principal balance"] = schedule_a_bal["Principal"].sum() - schedule_a_bal["Principal"].cumsum()

schedule_a_melt = schedule_a_bal.drop(0).melt(id_vars="Period", value_vars=["Principal balance", "Interest balance"],
                                             value_name="Amount($)", var_name="Remaining")

a_plot = alt.Chart(schedule_a_melt).mark_line()\
            .encode(x="Period", y=alt.Y("Amount($)"), color="Remaining")

a_plot

alt.Chart(...)

### 20 year fixed-rate mortgage

In [4]:
p = 1_000_000
i = 0.025
n = 20

mortgage_b = Mortgage(p, i, n)

for i in range(n*12):
    mortgage_b.MakePayment()
    
schedule_b = mortgage_b.AmortizationSchedule()

schedule_b

,Period,Payments,Principal,Interest,Unpaid Balance
0,0,0.00,0.00,0.00,1000000.00
1,1,5299.03,3215.70,2083.33,996784.30
2,2,5299.03,3222.39,2076.63,993561.91
3,3,5299.03,3229.11,2069.92,990332.80
4,4,5299.03,3235.84,2063.19,987096.97
...,...,...,...,...,...
236,236,5299.03,5244.17,54.85,21086.18
237,237,5299.03,5255.10,43.93,15831.08
238,238,5299.03,5266.05,32.98,10565.03
239,239,5299.03,5277.02,22.01,5288.01


In [5]:
schedule_b_bal = schedule_b.copy()
schedule_b_bal["Interest balance"] = schedule_b_bal["Interest"].sum() - schedule_b_bal["Interest"].cumsum()
schedule_b_bal["Principal balance"] = schedule_b_bal["Principal"].sum() - schedule_b_bal["Principal"].cumsum()

schedule_b_melt = schedule_b_bal.drop(0).melt(id_vars="Period", value_vars=["Principal balance", "Interest balance"],
                                             value_name="Amount($)", var_name="Remaining")

b_plot = alt.Chart(schedule_b_melt).mark_line()\
            .encode(x="Period", y=alt.Y("Amount($)", scale=alt.Scale(domain=[0, 1_400_000])), color="Remaining")\
            .properties(title = "20 Year Fixed rate Mortgage")

b_plot

alt.Chart(...)

### 30 year ARM

For the ARM, the reset rate of an year is computed as the average of the predicted rates for that year.

In [6]:
df = pd.read_csv("MORTGAGE30US.csv", parse_dates=["DATE"])[["DATE", "MORTGAGE30US"]]
sim_rates = df[df["DATE"] > "1991-01-01"]
sim_rates = sim_rates.groupby(sim_rates["DATE"].map(lambda x: x.year)).mean()
sim_rates.index.name = "YEAR"
sim_rates[-24:-1]

C:\Users\Abishek\AppData\Local\Temp\ipykernel_26472\169174269.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sim_rates = sim_rates.groupby(sim_rates["DATE"].map(lambda x: x.year)).mean()


,MORTGAGE30US
YEAR,
1999,7.44
2000,8.05
2001,6.97
2002,6.54
2003,5.83
2004,5.84
2005,5.87
2006,6.41
2007,6.34


In [7]:
p = 1_000_000
i = sim_rates["MORTGAGE30US"][1992] / 100
n = 30
var = 23
reset = 1
reset_rates = (sim_rates["MORTGAGE30US"].iloc[-24:-1] / 100).to_list()
reset_rates

mortgage_c = Mortgage(p, i, n, var, reset, reset_rates)

for i in range(n*12):
    mortgage_c.MakePayment()
    
schedule_c = mortgage_c.AmortizationSchedule()

schedule_c

,Period,Payments,Principal,Interest,Unpaid Balance
0,0,0.00,0.00,0.00,1000000.00
1,1,7611.45,619.62,6991.82,999380.38
2,2,7611.45,623.95,6987.49,998756.42
3,3,7611.45,628.32,6983.13,998128.11
4,4,7611.45,632.71,6978.74,997495.39
...,...,...,...,...,...
356,356,5606.14,5537.56,68.58,22287.06
357,357,5606.14,5551.21,54.93,16735.85
358,358,5606.14,5564.89,41.25,11170.96
359,359,5606.14,5578.61,27.53,5592.35


In [8]:
schedule_c_cal = schedule_c.copy()
schedule_c_cal["Interest balance"] = schedule_c_cal["Interest"].sum() - schedule_c_cal["Interest"].cumsum()
schedule_c_cal["Principal balance"] = schedule_c_cal["Principal"].sum() - schedule_c_cal["Principal"].cumsum()

schedule_c_melt = schedule_c_cal.drop(0).melt(id_vars="Period", value_vars=["Principal balance", "Interest balance"],
                                             value_name="Amount($)", var_name="Remaining")

c_plot = alt.Chart(schedule_c_melt).mark_line()\
            .encode(x="Period", y=alt.Y("Amount($)", scale=alt.Scale(domain=[0, 1_400_000])), color="Remaining")\
            .properties(title = "30 Year ARM")

c_plot

alt.Chart(...)